In [1]:
import json

In [2]:
dirques = r"./result/direction_sort/ques_sort.json"
with open(dirques, "r", encoding="utf8") as reader:
    dir_seq = [tuple(e) for e in json.load(reader)]

In [3]:
trii = r"./merged_tri_infos_m.json"
trii_dict = dict()
with open(trii, "r", encoding="utf8") as reader:
    data = json.load(reader)
    for e in data: trii_dict[(e["image_name"], e["question"])] = e

In [4]:
dump_contents = list()
for ds in dir_seq: dump_contents.append(trii_dict[ds])

In [5]:
# last = ""; jq = set(); qc = 0
# for ds in dir_seq:
#     if ds[0] != last: 
#         if ds[0] in jq: qc += 1
#         else: jq.add(ds[0])
#     last = ds[0]
# print(qc)

exc_list = [
    "拍摄者",
    "拍摄视角",
    "拍摄这张图片的人",
    "拍摄这张照片的人",
    "拍摄人",
    "当前拍摄",
    "拍照人",
    "拍照的人",
    "拍照者"
]
def is_alter_cam(ques:str) -> bool:
    return any([el in ques for el in exc_list])

from tqdm.notebook import tqdm

cc = 0
last_ref = dict(); last_in = ""
for i in tqdm(range(len(dump_contents))):
    dc = dump_contents[i]
    if "reference" in dc and dc["reference"]:
        ref = dc["reference"]
        if dc["image_name"] != last_in:
            last_ref = ref
            last_in = dc["image_name"]
        elif ref["ref"] == "画面视角":
            if is_alter_cam(dc["question"]): last_ref = ref
            else: dump_contents[i]["reference"] = last_ref; cc += 1
        else: last_ref = ref
print(cc)

  0%|          | 0/3897 [00:00<?, ?it/s]

263


In [6]:
dump_path = r"./dir_ques_tri_infos.json"
with open(dump_path, "w", encoding="utf8") as writer:
    json.dump(dump_contents, writer, indent=4, ensure_ascii=False)